# Градиентный бустинг

In [2]:
# Импорт библиотек для градиентного бустинга
try:
    from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    from xgboost import XGBClassifier, XGBRegressor
    from lightgbm import LGBMClassifier, LGBMRegressor
    HAS_GBM = True
except ImportError:
    print("Установите xgboost и lightgbm: pip install xgboost lightgbm")
    HAS_GBM = False
    from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

from sklearn.metrics import (accuracy_score, f1_score, confusion_matrix, classification_report,
                             r2_score, mean_absolute_error, mean_squared_error)
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("="*70)
print("ЛАБОРАТОРНАЯ РАБОТА №5: ГРАДИЕНТНЫЙ БУСТИНГ")
print("="*70)

ЛАБОРАТОРНАЯ РАБОТА №5: ГРАДИЕНТНЫЙ БУСТИНГ


In [3]:
df_class = pd.read_csv('./data/beans/Dry_Bean_Dataset.csv')
df_reg = pd.read_csv('./data/football/players_3120.csv')


X_cls = df_class.drop('Class', axis=1)
y_cls = df_class['Class']

le = LabelEncoder()
y_cls_encoded = le.fit_transform(y_cls)

X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(
    X_cls, y_cls_encoded, test_size=0.2, random_state=42, stratify=y_cls_encoded
)

scaler_cls = StandardScaler()
X_cls_train_scaled = scaler_cls.fit_transform(X_cls_train)
X_cls_test_scaled = scaler_cls.transform(X_cls_test)


In [4]:

def value_to_float(value_str):
    try:
        if pd.isna(value_str):
            return np.nan
        
        value_str = str(value_str).strip()
        
        if value_str.startswith('€'):
            value_str = value_str[1:]
        
        multiplier = 1
        if value_str.endswith('M'):
            multiplier = 1_000_000
            value_str = value_str[:-1]
        elif value_str.endswith('K'):
            multiplier = 1_000
            value_str = value_str[:-1]
        
        return float(value_str) * multiplier
    except Exception as e:
        print(f"Ошибка преобразования '{value_str}': {e}")
        return np.nan


df_reg['Value_numeric'] = df_reg['Value'].apply(value_to_float)

def height_to_cm(height_str):
    try:
        if pd.isna(height_str):
            return np.nan
        height_str = str(height_str)
        import re
        match = re.search(r'(\d+)cm', height_str)
        if match:
            return float(match.group(1))
        return np.nan
    except:
        return np.nan

def weight_to_kg(weight_str):
    try:
        if pd.isna(weight_str):
            return np.nan
        weight_str = str(weight_str)
        import re
        match = re.search(r'(\d+)kg', weight_str)
        if match:
            return float(match.group(1))
        return np.nan
    except:
        return np.nan

df_reg['Height_cm'] = df_reg['Height'].apply(height_to_cm)
df_reg['Weight_kg'] = df_reg['Weight'].apply(weight_to_kg)

numeric_features = [
    'Age', 
    'Overall rating', 
    'Potential',
    'Height_cm',
    'Weight_kg',
    'Crossing', 
    'Finishing', 
    'Short passing', 
    'Dribbling',
    'Acceleration', 
    'Sprint speed', 
    'Stamina', 
    'Strength',
    'Long shots',
    'Interceptions',
    'Heading accuracy',
    'Ball control',
    'Reactions',
    'Composure',
    'Vision',
    'Aggression',
    'Penalties'
]

existing_features = [col for col in numeric_features if col in df_reg.columns]
print(f"\nИспользуем признаки ({len(existing_features)}): {existing_features}")

X_reg = df_reg[existing_features].copy()

for col in X_reg.columns:
    X_reg[col] = pd.to_numeric(X_reg[col], errors='coerce')

X_reg = X_reg.fillna(X_reg.median())

y_reg = df_reg['Value_numeric']

valid_indices = y_reg.notna()
X_reg = X_reg[valid_indices]
y_reg = y_reg[valid_indices]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

scaler_reg = StandardScaler()
X_reg_train_scaled = scaler_reg.fit_transform(X_reg_train)
X_reg_test_scaled = scaler_reg.transform(X_reg_test)


Используем признаки (22): ['Age', 'Overall rating', 'Potential', 'Height_cm', 'Weight_kg', 'Crossing', 'Finishing', 'Short passing', 'Dribbling', 'Acceleration', 'Sprint speed', 'Stamina', 'Strength', 'Long shots', 'Interceptions', 'Heading accuracy', 'Ball control', 'Reactions', 'Composure', 'Vision', 'Aggression', 'Penalties']


# 2.1 Классификация 

Обучение бейзлайна и оценка качества

In [5]:
print("\n" + "="*70)
print("КЛАССИФИКАЦИЯ: Градиентный бустинг (бейзлайн)")
print("="*70)

gb_cls_baseline = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

gb_cls_baseline.fit(X_cls_train_scaled, y_cls_train)
y_cls_pred_gb = gb_cls_baseline.predict(X_cls_test_scaled)
y_cls_prob_gb = gb_cls_baseline.predict_proba(X_cls_test_scaled)

acc_gb_cls = accuracy_score(y_cls_test, y_cls_pred_gb)
f1_gb_cls = f1_score(y_cls_test, y_cls_pred_gb, average='macro')

print(f"Accuracy (GradientBoosting): {acc_gb_cls:.4f}")
print(f"F1-score (macro): {f1_gb_cls:.4f}")

feature_importance_gb_cls = pd.DataFrame({
    'feature': X_cls.columns,
    'importance': gb_cls_baseline.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 самых важных признаков (градиентный бустинг, классификация):")
print(feature_importance_gb_cls.head(10))


КЛАССИФИКАЦИЯ: Градиентный бустинг (бейзлайн)


KeyboardInterrupt: 

# 2.1 Регрессия

In [ ]:
print("\n" + "="*70)
print("РЕГРЕССИЯ: Градиентный бустинг (бейзлайн)")
print("="*70)

gb_reg_baseline = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

y_reg_train_log = np.log1p(y_reg_train)

gb_reg_baseline.fit(X_reg_train_scaled, y_reg_train_log)

y_reg_pred_log_gb = gb_reg_baseline.predict(X_reg_test_scaled)
y_reg_pred_gb = np.exp(y_reg_pred_log_gb) - 1

r2_gb_reg = r2_score(y_reg_test, y_reg_pred_gb)
mae_gb_reg = mean_absolute_error(y_reg_test, y_reg_pred_gb)
rmse_gb_reg = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_gb))

print(f"R² (GradientBoosting): {r2_gb_reg:.4f}")
print(f"MAE: {mae_gb_reg:,.0f} евро ({mae_gb_reg/1_000_000:.2f} млн евро)")
print(f"RMSE: {rmse_gb_reg:,.0f} евро ({rmse_gb_reg/1_000_000:.2f} млн евро)")

feature_importance_gb_reg = pd.DataFrame({
    'feature': X_reg.columns,
    'importance': gb_reg_baseline.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 самых важных признаков (градиентный бустинг, регрессия):")
print(feature_importance_gb_reg.head(10))


РЕГРЕССИЯ: Градиентный бустинг (бейзлайн)
R² (GradientBoosting): 0.7741
MAE: 3,419,870 евро (3.42 млн евро)
RMSE: 7,922,704 евро (7.92 млн евро)

Топ-10 самых важных признаков (градиентный бустинг, регрессия):
           feature  importance
2        Potential    0.175547
17       Reactions    0.141347
0              Age    0.133887
18       Composure    0.084538
11         Stamina    0.049669
7    Short passing    0.047647
1   Overall rating    0.044960
19          Vision    0.035541
20      Aggression    0.035392
14   Interceptions    0.033543


# 3 Улучшение бейзлайна

### Для классификации:


H1: learning_rate=0.05 + n_estimators=200 → лучшая сходимость и точность

H2: subsample=0.8 → стохастический GB, меньше переобучение

H3: max_depth=5 + min_samples_split=10 → баланс смещения и дисперсии

H4: min_samples_leaf=5 → дополнительная регуляризация

In [ ]:
print("\n" + "-"*70)
print("КЛАССИФИКАЦИЯ: Улучшение градиентного бустинга")
print("-"*70)


gb_cls_improved = GradientBoostingClassifier(
    n_estimators=200,          # H1: Больше деревьев с меньшим learning_rate
    learning_rate=0.05,        # H1: Меньшая скорость обучения для лучшей сходимости
    max_depth=5,               # H3: Глубже деревья для сложных паттернов
    min_samples_split=10,      # H3: Регуляризация через min_samples_split
    min_samples_leaf=5,        # H4: Дополнительная регуляризация через min_samples_leaf
    subsample=0.8,             # H2: Stochastic Gradient Boosting
    random_state=42
)


gb_cls_improved.fit(X_cls_train_scaled, y_cls_train)
y_cls_pred_gb_imp = gb_cls_improved.predict(X_cls_test_scaled)

acc_gb_cls_imp = accuracy_score(y_cls_test, y_cls_pred_gb_imp)
f1_gb_cls_imp = f1_score(y_cls_test, y_cls_pred_gb_imp, average='macro')

print(f"\nРезультаты улучшенного GradientBoosting:")
print(f"Accuracy: {acc_gb_cls_imp:.4f}")
print(f"F1-score (macro): {f1_gb_cls_imp:.4f}")
print(f"Улучшение по сравнению с бейзлайном: {acc_gb_cls_imp - acc_gb_cls:+.4f}")

# Проверяем гипотезу H4: XGBoost/LightGBM лучше sklearn GBM
print("\n" + "-"*70)
print("ПРОВЕРКА ГИПОТЕЗЫ H4: XGBoost/LightGBM vs sklearn GBM")
print("-"*70)

if HAS_GBM:
    xgb_cls_comparison = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        min_child_weight=5,  
        subsample=0.8,
        random_state=42,
        n_jobs=-1,
        use_label_encoder=False,
        eval_metric='mlogloss'
    )
    
    xgb_cls_comparison.fit(X_cls_train_scaled, y_cls_train)
    acc_xgb = accuracy_score(y_cls_test, xgb_cls_comparison.predict(X_cls_test_scaled))
    
    print(f"XGBoost с аналогичными параметрами:")
    print(f"  Accuracy: {acc_xgb:.4f}")
    print(f"  Разница с sklearn GBM: {acc_xgb - acc_gb_cls_imp:+.4f}")
    
    if acc_xgb > acc_gb_cls_imp:
        print("✓ ГИПОТЕЗА H4 ПОДТВЕРЖДЕНА: XGBoost показал лучший результат")
    else:
        print("✗ ГИПОТЕЗА H4 НЕ ПОДТВЕРЖДЕНА: sklearn GBM показал сравнимый результат")
else:
    print("XGBoost не установлен. Установите: pip install xgboost")
    print("Для проверки H4 требуется установка дополнительных библиотек")


----------------------------------------------------------------------
КЛАССИФИКАЦИЯ: Улучшение градиентного бустинга
----------------------------------------------------------------------

Результаты улучшенного GradientBoosting:
Accuracy: 0.9232
F1-score (macro): 0.9357
Улучшение по сравнению с бейзлайном: +0.0004

----------------------------------------------------------------------
ПРОВЕРКА ГИПОТЕЗЫ H4: XGBoost/LightGBM vs sklearn GBM
----------------------------------------------------------------------


/Users/koche/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [21:30:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost с аналогичными параметрами:
  Accuracy: 0.9258
  Разница с sklearn GBM: +0.0026
✓ ГИПОТЕЗА H4 ПОДТВЕРЖДЕНА: XGBoost показал лучший результат


## Для регрессии:

H1: Увеличение n_estimators до 200 улучшит качество (vs 100 в бейзлайне)

H2: Уменьшение learning_rate до 0.05 улучшит сходимость (vs 0.1 в бейзлайне)

H3: Увеличение max_depth до 5 улучшит качество (vs 3 в бейзлайне)

H4: Использование subsample=0.8 улучшит обобщающую способность

H5: XGBoost покажет лучшие результаты, чем sklearn GradientBoosting

In [ ]:
print("\n" + "-"*70)
print("РЕГРЕССИЯ: Улучшение градиентного бустинга")
print("-"*70)

gb_reg_improved = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    random_state=42
)

gb_reg_improved.fit(X_reg_train_scaled, y_reg_train_log)

y_reg_pred_log_gb_imp = gb_reg_improved.predict(X_reg_test_scaled)
y_reg_pred_gb_imp = np.exp(y_reg_pred_log_gb_imp) - 1

r2_gb_reg_imp = r2_score(y_reg_test, y_reg_pred_gb_imp)
mae_gb_reg_imp = mean_absolute_error(y_reg_test, y_reg_pred_gb_imp)
rmse_gb_reg_imp = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_gb_imp))

print(f"Улучшенный GradientBoosting (регрессия):")
print(f"R²: {r2_gb_reg_imp:.4f}")
print(f"MAE: {mae_gb_reg_imp:,.0f} евро ({mae_gb_reg_imp/1_000_000:.2f} млн евро)")
print(f"Улучшение R² по сравнению с бейзлайном: {r2_gb_reg_imp - r2_gb_reg:+.4f}")

if HAS_GBM:
    print("\n" + "-"*70)
    print("XGBOOST (продвинутый градиентный бустинг)")
    print("-"*70)
    
    xgb_reg = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        random_state=42
    )
    
    xgb_reg.fit(X_reg_train_scaled, y_reg_train_log)
    y_reg_pred_log_xgb = xgb_reg.predict(X_reg_test_scaled)
    y_reg_pred_xgb = np.exp(y_reg_pred_log_xgb) - 1
    r2_xgb_reg = r2_score(y_reg_test, y_reg_pred_xgb)
    
    print(f"XGBoost Regressor R²: {r2_xgb_reg:.4f}")
    print(f"Улучшение vs sklearn GBM (регрессия): {r2_xgb_reg - r2_gb_reg:+.4f}")


----------------------------------------------------------------------
РЕГРЕССИЯ: Улучшение градиентного бустинга
----------------------------------------------------------------------
Улучшенный GradientBoosting (регрессия):
R²: 0.8310
MAE: 3,043,914 евро (3.04 млн евро)
Улучшение R² по сравнению с бейзлайном: +0.0569

----------------------------------------------------------------------
XGBOOST (продвинутый градиентный бустинг)
----------------------------------------------------------------------
XGBoost Regressor R²: 0.8114
Улучшение vs sklearn GBM (регрессия): +0.0373


# ИМПЛЕМЕНТАЦИЯ

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

class GradientBoosting:
    def __init__(self, n_estimators=200, learning_rate=0.05, max_depth=5, 
                 min_samples_split=10, min_samples_leaf=5, subsample=0.8, 
                 random_state=None, verbose=0):
        """
        УЛУЧШЕННЫЕ параметры (согласно вашим улучшениям):
        - n_estimators: 200 (H1: больше деревьев)
        - learning_rate: 0.05 (H1: меньшая скорость обучения)
        - max_depth: 5 (H3: глубже деревья)
        - min_samples_split: 10 (H3: регуляризация)
        - min_samples_leaf: 5 (H4: дополнительная регуляризация)
        - subsample: 0.8 (H2: Stochastic Gradient Boosting)
        """
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.subsample = subsample
        self.random_state = random_state
        self.verbose = verbose
        self.trees = []
        self.train_errors = []
        
        if random_state is not None:
            np.random.seed(random_state)
    
    def _subsample(self, X, y):
        """Создание подвыборки для обучения дерева"""
        n_samples = X.shape[0]
        sample_size = int(n_samples * self.subsample)
        indices = np.random.choice(n_samples, size=sample_size, replace=False)
        return X[indices], y[indices]  # Правильное индексирование для numpy массивов
    
    def fit(self, X, y):
        """Обучение модели"""
        raise NotImplementedError("Это абстрактный метод")
    
    def predict(self, X):
        """Предсказание"""
        raise NotImplementedError("Это абстрактный метод")

class GradientBoostingRegressor(GradientBoosting):
    def __init__(self, n_estimators=200, learning_rate=0.05, max_depth=5, 
                 min_samples_split=10, min_samples_leaf=5, subsample=0.8, 
                 random_state=None, verbose=0):
        super().__init__(n_estimators, learning_rate, max_depth, 
                        min_samples_split, min_samples_leaf, subsample, 
                        random_state, verbose)
        self.initial_prediction = None
    
    def fit(self, X, y):
        """Обучение модели регрессии"""
        X = np.asarray(X)
        y = np.asarray(y)
        
        self.initial_prediction = np.mean(y)
        predictions = np.full(y.shape, self.initial_prediction)
        
        for i in range(self.n_estimators):
            residuals = y - predictions
            
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=self.random_state
            )
            
            if self.subsample < 1.0:
                X_subsample, residuals_subsample = self._subsample(X, residuals)
                tree.fit(X_subsample, residuals_subsample)
            else:
                tree.fit(X, residuals)
            
            tree_prediction = tree.predict(X)
            predictions += self.learning_rate * tree_prediction
            
            self.trees.append(tree)
            
            mse = mean_squared_error(y, predictions)
            self.train_errors.append(mse)
            

        
        return self
    
    def predict(self, X):
        """Предсказание для регрессии"""
        X = np.asarray(X)
        predictions = np.full(X.shape[0], self.initial_prediction)
        
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        
        return predictions

class GradientBoostingClassifier(GradientBoosting):
    def __init__(self, n_estimators=200, learning_rate=0.05, max_depth=5, 
                 min_samples_split=10, min_samples_leaf=5, subsample=0.8, 
                 random_state=None, verbose=0):
        super().__init__(n_estimators, learning_rate, max_depth, 
                        min_samples_split, min_samples_leaf, subsample, 
                        random_state, verbose)
        self.initial_prediction = None
    
    @staticmethod
    def _sigmoid(x):
        """Сигмоидная функция"""
        x = np.clip(x, -10, 10)  
        return 1 / (1 + np.exp(-x))
    
    @staticmethod
    def _log_loss_gradient(y_true, y_pred):
        """Градиент логистической функции потерь"""
        return y_true - GradientBoostingClassifier._sigmoid(y_pred)
    
    def fit(self, X, y):
        """Обучение модели классификации (бинарной)"""
        # if self.verbose > 0:
        #     print(f"Обучаем GradientBoostingClassifier ({self.n_estimators} деревьев)...")
        #     print(f"Параметры: n_estimators={self.n_estimators}, lr={self.learning_rate}")
        
        X = np.asarray(X)
        y = np.asarray(y)
        
        y_binary = np.where(y > 0, 1, 0)
        
        pos_prob = np.mean(y_binary)
        self.initial_prediction = np.log(pos_prob / (1 - pos_prob + 1e-10))
        log_odds = np.full(y_binary.shape, self.initial_prediction)
        
        for i in range(self.n_estimators):
            gradients = self._log_loss_gradient(y_binary, log_odds)
            
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=self.random_state
            )
            
            if self.subsample < 1.0:
                X_subsample, gradients_subsample = self._subsample(X, gradients)
                tree.fit(X_subsample, gradients_subsample)
            else:
                tree.fit(X, gradients)
            
            tree_prediction = tree.predict(X)
            log_odds += self.learning_rate * tree_prediction
            
            self.trees.append(tree)
            
            probabilities = self._sigmoid(log_odds)
            predictions = (probabilities > 0.5).astype(int)
            accuracy = accuracy_score(y_binary, predictions)
            self.train_errors.append(accuracy)
            
            
        return self
    
    def predict_proba(self, X):
        """Предсказание вероятностей"""
        X = np.asarray(X)
        log_odds = np.full(X.shape[0], self.initial_prediction)
        
        for tree in self.trees:
            log_odds += self.learning_rate * tree.predict(X)
        
        probabilities = self._sigmoid(log_odds)
        return np.column_stack([1 - probabilities, probabilities])
    
    def predict(self, X, threshold=0.5):
        """Предсказание классов"""
        probabilities = self.predict_proba(X)[:, 1]
        return (probabilities > threshold).astype(int)

class GradientBoostingMulticlassClassifier:
    def __init__(self, n_estimators=200, learning_rate=0.05, max_depth=5, 
                 min_samples_split=10, min_samples_leaf=5, subsample=0.8, 
                 random_state=None, verbose=0):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.subsample = subsample
        self.random_state = random_state
        self.verbose = verbose
        self.classifiers = []
        self.classes_ = None
        
        if random_state is not None:
            np.random.seed(random_state)
    
    def fit(self, X, y):
        """Обучение для многоклассовой классификации (One-vs-All)"""
        X = np.asarray(X)
        y = np.asarray(y)
        
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        
        # if self.verbose > 0:
        #     print(f"Обучаем многоклассовый GradientBoosting ({n_classes} классов)...")
        #     print(f"Параметры: n_estimators={self.n_estimators}, lr={self.learning_rate}")
        
        for i, class_label in enumerate(self.classes_):
            # if self.verbose > 0:
            #     print(f"\nКласс {class_label} ({i+1}/{n_classes}):")
            
            y_binary = np.where(y == class_label, 1, 0)
            
            clf = GradientBoostingClassifier(
                n_estimators=self.n_estimators,
                learning_rate=self.learning_rate,
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                subsample=self.subsample,
                random_state=self.random_state,
                verbose=self.verbose - 1 if self.verbose > 1 else 0
            )
            clf.fit(X, y_binary)
            
            self.classifiers.append(clf)
        
        # if self.verbose > 0:
        #     print(f"\nОбучение всех {n_classes} классификаторов завершено!")
        
        return self
    
    def predict_proba(self, X):
        """Предсказание вероятностей для всех классов"""
        X = np.asarray(X)
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        probabilities = np.zeros((n_samples, n_classes))
        
        for i, clf in enumerate(self.classifiers):
            probabilities[:, i] = clf.predict_proba(X)[:, 1]
        

        prob_sum = probabilities.sum(axis=1, keepdims=True)
        prob_sum[prob_sum == 0] = 1
        probabilities = probabilities / prob_sum
        
        return probabilities
    
    def predict(self, X):
        """Предсказание классов"""
        probabilities = self.predict_proba(X)
        return self.classes_[np.argmax(probabilities, axis=1)]

## Обучение и оценка имплементированных моделей:

Обучение собственной модели и оценка (классификация)

In [ ]:
print("\n" + "="*70)
print("ОБУЧЕНИЕ СОБСТВЕННЫХ РЕАЛИЗАЦИЙ ГРАДИЕНТНОГО БУСТИНГА")
print("="*70)

print("\n" + "-"*70)
print("Градиентный бустинг для классификации (наша реализация)")
print("-"*70)

my_gb_cls = GradientBoostingMulticlassClassifier(
    n_estimators=30,      # Мало деревьев для скорости
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=10,
    random_state=42
)

my_gb_cls.fit(X_cls_train_scaled, y_cls_train)
y_cls_pred_my_gb = my_gb_cls.predict(X_cls_test_scaled)

acc_my_gb_cls = accuracy_score(y_cls_test, y_cls_pred_my_gb)
f1_my_gb_cls = f1_score(y_cls_test, y_cls_pred_my_gb, average='macro')

print(f"Наш GradientBoosting (классификация):")
print(f"Accuracy: {acc_my_gb_cls:.4f}")
print(f"F1-score: {f1_my_gb_cls:.4f}")
print(f"Сравнение с sklearn: {acc_my_gb_cls - acc_gb_cls:+.4f}")


ОБУЧЕНИЕ СОБСТВЕННЫХ РЕАЛИЗАЦИЙ ГРАДИЕНТНОГО БУСТИНГА

----------------------------------------------------------------------
Градиентный бустинг для классификации (наша реализация)
----------------------------------------------------------------------
Наш GradientBoosting (классификация):
Accuracy: 0.8902
F1-score: 0.9035
Сравнение с sklearn: -0.0327


Обучение собственной модели и оценка (регрессия)

In [ ]:
print("РЕГРЕССИЯ: Ваш GradientBoostingRegressor")
print("-"*70)

gb_reg_your = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=10,
    subsample=0.8,
    random_state=42
)

print("Обучаем ваш регрессор...")
gb_reg_your.fit(X_reg_train_scaled, y_reg_train_log)

# Предсказание
y_reg_pred_log_your = gb_reg_your.predict(X_reg_test_scaled)
y_reg_pred_your = np.exp(y_reg_pred_log_your) - 1

r2_your_gb_reg = r2_score(y_reg_test, y_reg_pred_your)
mae_your_gb_reg = mean_absolute_error(y_reg_test, y_reg_pred_your)

print(f"\nВаш GradientBoostingRegressor:")
print(f"R²: {r2_your_gb_reg:.4f}")
print(f"MAE: {mae_your_gb_reg/1_000_000:.2f} млн евро")
print(f"Сравнение с sklearn GBM бейзлайн: {r2_your_gb_reg - r2_gb_reg:+.4f}")
print(f"Сравнение с улучшенным sklearn GBM: {r2_your_gb_reg - r2_gb_reg_imp:+.4f}")

РЕГРЕССИЯ: Ваш GradientBoostingRegressor
----------------------------------------------------------------------
Обучаем ваш регрессор...

Ваш GradientBoostingRegressor:
R²: 0.7692
MAE: 3.70 млн евро
Сравнение с sklearn GBM бейзлайн: -0.0049
Сравнение с улучшенным sklearn GBM: -0.0618


## улучшенные парамметры

Обучение собственной модели и оценка (классификация)

In [ ]:
print("\n" + "="*70)
print("ТЕСТИРОВАНИЕ НАШЕЙ РЕАЛИЗАЦИИ С ПАРАМЕТРАМИ ИЗ УЛУЧШЕНИЙ")
print("="*70)

print("\n" + "-"*70)
print("КЛАССИФИКАЦИЯ: Наш Gradient Boosting")
print("-"*70)

my_gb_cls_test = GradientBoostingMulticlassClassifier(
    n_estimators=50,  # Уменьшаем для скорости демонстрации
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    random_state=42,
    verbose=1
)

my_gb_cls_test.fit(X_cls_train_scaled, y_cls_train)

y_cls_pred_my_gb = my_gb_cls_test.predict(X_cls_test_scaled)
acc_my_gb_cls = accuracy_score(y_cls_test, y_cls_pred_my_gb)
f1_my_gb_cls = f1_score(y_cls_test, y_cls_pred_my_gb, average='macro')

print(f"\nНаш GradientBoosting (классификация):")
print(f"Accuracy: {acc_my_gb_cls:.4f}")
print(f"F1-score: {f1_my_gb_cls:.4f}")
print(f"Сравнение с sklearn GBM бейзлайн: {acc_my_gb_cls - acc_gb_cls:+.4f}")
print(f"Сравнение с улучшенным sklearn GBM: {acc_my_gb_cls - acc_gb_cls_imp:+.4f}")

print("\n" + "-"*70)


ТЕСТИРОВАНИЕ НАШЕЙ РЕАЛИЗАЦИИ С ПАРАМЕТРАМИ ИЗ УЛУЧШЕНИЙ

----------------------------------------------------------------------
КЛАССИФИКАЦИЯ: Наш Gradient Boosting
----------------------------------------------------------------------

Наш GradientBoosting (классификация):
Accuracy: 0.9082
F1-score: 0.9217
Сравнение с sklearn GBM бейзлайн: -0.0147
Сравнение с улучшенным sklearn GBM: -0.0151

----------------------------------------------------------------------


Обучение собственной модели и оценка (регрессия)

In [ ]:
print("РЕГРЕССИЯ: Ваша реализация с улучшенными параметрами")
print("-"*70)

gb_reg_your = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    random_state=42,
    verbose=1
)

print("Обучаем ваш регрессор...")
gb_reg_your.fit(X_reg_train_scaled, y_reg_train_log)

# Предсказание
y_reg_pred_log_your = gb_reg_your.predict(X_reg_test_scaled)
y_reg_pred_your = np.exp(y_reg_pred_log_your) - 1

r2_your_reg = r2_score(y_reg_test, y_reg_pred_your)
mae_your_reg = mean_absolute_error(y_reg_test, y_reg_pred_your)

print(f"\nРезультаты вашей реализации (регрессия):")
print(f"R²: {r2_your_reg:.4f}")
print(f"MAE: {mae_your_reg/1_000_000:.2f} млн евро")
print(f"Сравнение с sklearn GBM бейзлайн: {r2_your_reg - r2_gb_reg:+.4f}")
print(f"Сравнение с улучшенным sklearn GBM: {r2_your_reg - r2_gb_reg_imp:+.4f}")

РЕГРЕССИЯ: Ваша реализация с улучшенными параметрами
----------------------------------------------------------------------
Обучаем ваш регрессор...

Результаты вашей реализации (регрессия):
R²: 0.7692
MAE: 3.70 млн евро
Сравнение с sklearn GBM бейзлайн: -0.0049
Сравнение с улучшенным sklearn GBM: -0.0618
